In [1]:
import tensorflow.compat.v2 as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow_model_optimization as tfmot
import numpy as np
from qkeras import print_qstats
from qkeras import QActivation
from qkeras import QDense
from qkeras import quantized_bits
from qkeras import ternary
from qkeras.estimate import print_qstats
from qkeras.estimate import extract_model_operations
from qkeras.utils import quantized_model_debug
from qkeras.utils import model_save_quantized_weights
from qkeras.utils import model_quantize
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import tempfile
import os
import sys
import csv
import zipfile
import math
import time

In [2]:
# Reproducibility
seed = 9 # arbitrary number
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

In [3]:
N_channels = 40
data_path = os.getcwd()
print(data_path)

/Users/jackkelly/Desktop/Submission


In [4]:
### IMPORT, SHUFFLE AND SCALE DATA ###
def processing_data(N_channels, file_path):
    df = pd.read_csv(file_path)
    # dataset
    dataset = df.values
    # input power and channels statuses
    X = dataset[:,0:N_channels+1]
    # channel SNRs
    Y = dataset[:,N_channels+1:2*N_channels+1]
    # shuffle necessary because while shuffle=True in .fit() shuffles the data, it splits into different datasets first
    X, Y = shuffle(X, Y)
    # scaling between 0 and 1
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    Y = min_max_scaler.fit_transform(Y)
    return X, Y

In [5]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)

In [6]:
file_path = f'{data_path}/Data/10000_initial_data.csv'
X_scale, Y_scale = processing_data(N_channels, file_path)
file_path = f'{data_path}/Data/10000_initial_data_testing.csv'
X_test, Y_test = processing_data(N_channels, file_path)
model = keras.models.load_model(f'{data_path}/Tuned_ANN_model.h5')
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
batch_size = 32
epochs = 150
validation_split = 0.2

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 140)               5880      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               14100     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                4040      
Total params: 24,020
Trainable params: 24,020
Non-trainable params: 0
_________________________________________________________________


In [8]:
def prune_and_quantize(desired_sparsity, quantized_bit_number):
    file_path = f'{data_path}/Data/10000_initial_data.csv'
    X_scale, Y_scale = processing_data(N_channels, file_path)
    file_path = f'{data_path}/Data/10000_initial_data_testing.csv'
    X_test, Y_test = processing_data(N_channels, file_path)
    model = keras.models.load_model(f'{data_path}/Tuned_ANN_model.h5')
    print('---------------------------')
    print(f'Desired sparsity: {desired_sparsity}, Quantized bit number: {quantized_bit_number}')
    config = {
      "QDense": {
          "kernel_quantizer": f"quantized_bits({quantized_bit_number},0,1)",
          "bias_quantizer": f"quantized_bits({quantized_bit_number})"
      },
      "QActivation": { "relu": f"quantized_bits({quantized_bit_number})" },
    }
    qmodel = model_quantize(model, config, 16, transfer_weights=True)
    qmodel.summary()
    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=desired_sparsity, begin_step=0, end_step=-1, frequency=200)
    }
    pruned_q_model = prune_low_magnitude(qmodel, **pruning_params)
    # 'prune_low_magnitude' requires a recompile.
    pruned_q_model.compile(optimizer=keras.optimizers.Adam(),
                        loss=keras.losses.MeanSquaredError(), metrics=['mean_squared_error'])
    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
    ]
    pruned_q_model.fit(X_scale, Y_scale,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=callbacks, verbose=0)
    eval_pruned_model = pruned_q_model.evaluate(X_test, Y_test, verbose=0)[1]
    print(f'Pruned model MSE: {eval_pruned_model}')
    qmodel_stripped = tfmot.sparsity.keras.strip_pruning(pruned_q_model)
    print_qstats(qmodel_stripped)
    _, pruned_keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(qmodel_stripped, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)
    # Size of model after pruning
    size = get_gzipped_model_size(pruned_keras_file)
    print("Size of gzipped baseline Keras model: %.2f bytes" % (size))
    print('---------------------------')
    return eval_pruned_model, size, qmodel_stripped

In [9]:
# baseline model size
model = keras.models.load_model(f'{data_path}/Tuned_ANN_model.h5')
_, baseline_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, baseline_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', baseline_keras_file)
# Size of model after pruning
baseline_size = get_gzipped_model_size(baseline_keras_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (baseline_size))

Saved pruned Keras model to: /var/folders/yp/nbwcv8453t3f3sx3xxwwycqm0000gn/T/tmpqbhhfu6k.h5
Size of gzipped baseline Keras model: 91005.00 bytes


In [10]:
size_results = []
model_performance = []
quantized_bit_numbers = [4, 8, 12, 16]
sparsity_list = [0.2, 0.4, 0.6, 0.8]
for i in quantized_bit_numbers:
    for j in sparsity_list:
        eval_pruned_model, size, qmodel_stripped = prune_and_quantize(j, i)
        model_performance.append(eval_pruned_model)
        size_results.append(size)

---------------------------
Desired sparsity: 0.2, Quantized bit number: 4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (QDense)               (None, 140)               5880      
_________________________________________________________________
dense_1 (QDense)             (None, 100)               14100     
_________________________________________________________________
dense_2 (QDense)             (None, 40)                4040      
Total params: 24,020
Trainable params: 24,020
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Please use `layer.add_weight` method instead.
Pruned model MSE: 0.004524689633399248
Instructions for updating:
Use ref() instead.

Number of operations in model:
    dense                         : 5740  (smult_4_8)
    dense_1                       : 14000 (smult_4_16)
    dens

---------------------------
Desired sparsity: 0.2, Quantized bit number: 8
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (QDense)               (None, 140)               5880      
_________________________________________________________________
dense_1 (QDense)             (None, 100)               14100     
_________________________________________________________________
dense_2 (QDense)             (None, 40)                4040      
Total params: 24,020
Trainable params: 24,020
Non-trainable params: 0
_________________________________________________________________
Pruned model MSE: 0.0010846592485904694

Number of operations in model:
    dense                         : 5740  (smult_8_8)
    dense_1                       : 14000 (smult_8_16)
    dense_2                       : 4000  (smult_8_16)

Number of operation types in model:
    smult_8_16                    

Pruned model MSE: 0.0006227016565389931

Number of operations in model:
    dense                         : 5740  (smult_12_8)
    dense_1                       : 14000 (smult_12_16)
    dense_2                       : 4000  (smult_12_16)

Number of operation types in model:
    smult_12_16                   : 18000
    smult_12_8                    : 5740

Weight profiling:
    dense_weights                  : 5740  (12-bit unit)
    dense_bias                     : 140   (12-bit unit)
    dense_1_weights                : 14000 (12-bit unit)
    dense_1_bias                   : 100   (12-bit unit)
    dense_2_weights                : 4000  (12-bit unit)
    dense_2_bias                   : 40    (12-bit unit)

Weight sparsity:
... quantizing model
    dense                          : 0.1956
    dense_1                        : 0.1988
    dense_2                        : 0.1983
    ----------------------------------------
    Total Sparsity                 : 0.1979
Saved pruned Keras m

---------------------------
Desired sparsity: 0.4, Quantized bit number: 16
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (QDense)               (None, 140)               5880      
_________________________________________________________________
dense_1 (QDense)             (None, 100)               14100     
_________________________________________________________________
dense_2 (QDense)             (None, 40)                4040      
Total params: 24,020
Trainable params: 24,020
Non-trainable params: 0
_________________________________________________________________
Pruned model MSE: 0.0008951331255957484

Number of operations in model:
    dense                         : 5740  (smult_16_8)
    dense_1                       : 14000 (smult_16_16)
    dense_2                       : 4000  (smult_16_16)

Number of operation types in model:
    smult_16_16               

In [ ]:
print(size_results)
print(model_performance[0:4])
print(model_performance[4:8])
print(model_performance[8:12])
print(model_performance[12:16])

# Model sizes (bytes)

In [ ]:
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
plt.figure(figsize=(13,6))
x = ['0.2','0.4','0.6','0.8']
quantized_bit_numbers = [4, 8, 12, 16]

values = np.array(size_results)
idx = []
for i in quantized_bit_numbers:
    for sp in x:
        idx.append(f'{sp} ({i})')
    
idx = np.array(idx)
clrs = ['blue']*4 + ['green']*4 + ['red']*4 + ['black']*4
plt.bar(idx, values, color=clrs, width=0.6)
plt.ylabel('Size (bytes)')
plt.xlabel('Sparsity (Quantized bits width)')
# plt.axhline(y=baseline_size, linestyle='-.', linewidth=1, color='k')
# plt.ylim([0,105000])

colors = {'Quantized bit width = 4':'blue', 'Quantized bits width = 8':'green', 'Quantized bits width = 12':'red', 'Quantized bits width = 16':'black'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels)

plt.savefig(f'{data_path}/Figures/PQ/sizes.svg', bbox_inches='tight', format='svg', dpi=1200)
plt.show()

# Performance

In [ ]:
# setting up baseline non-quantized model's performance
baseline_model_performance = [model.evaluate(X_test, Y_test, verbose=1)[1]]*4
q4 = model_performance[0:4]
q8 = model_performance[4:8]
q12 = model_performance[8:12]
q16 = model_performance[12:16]
plt.figure()
plt.plot(sparsity_list, q4, 'x-', label='Quantized bits width = 4', color='blue')
plt.plot(sparsity_list, q8, 'x-', label='Quantized bits width = 8', color='green')
plt.plot(sparsity_list, q12, 'x-', label='Quantized bits width = 12', color='red')
plt.plot(sparsity_list, q16, 'x-', label='Quantized bits width = 16', color='black')
plt.plot(sparsity_list, baseline_model_performance, '-.', label='Baseline Model', color='orange')
plt.ylabel('MSE')
plt.xlabel('Sparsity')
plt.legend()
plt.savefig(f'{data_path}/Figures/PQ/pruning_and_quantization_MSE_epochs.svg', bbox_inches='tight', format='svg', dpi=1200)
plt.show()

# Bit operations (BOPs)

In [ ]:
# using formula
def bop_calculator(n, m, bw, ba, fp):
    num = m*n*((1-fp)*ba*bw + ba + bw + math.log(n, 2))
    return num

# where n (m) is the number of inputs (outputs),
# bw (ba) is the bit width of the weights (activations),
# and fp is the fraction of pruned layer weights

In [ ]:
# total_BOPs = 0
# total_BOPs += bop_calculator(41, 41, 8, 8, 0.6)
# total_BOPs += bop_calculator(41*140, 140, 8, 8, 0.6)
# total_BOPs += bop_calculator(140*100, 100, 8, 8, 0.6)
# total_BOPs += bop_calculator(100*40, 40, 8, 32, 0.6)
# print(total_BOPs)

# newtork: 41 -> 140 -> 100 -> 40

In [ ]:
sparsity_list = [0.2, 0.4, 0.6, 0.8]
quantized_bit_numbers = [4, 8, 12, 16]
bit_operations_comparison = []
for q_width in quantized_bit_numbers:
    for sparsity in sparsity_list:
        total_BOPs = 0
# #         is this definitely needed?
#         total_BOPs += bop_calculator(41, 41, q_width, q_width, sparsity)
        total_BOPs += bop_calculator(41*140, 140, q_width, q_width, sparsity)
        total_BOPs += bop_calculator(140*100, 100, q_width, q_width, sparsity)
        total_BOPs += bop_calculator(100*40, 40, q_width, 16, sparsity)
        print(f'Quantized to {q_width} bits with a model sparsity of {sparsity}: {total_BOPs}')
        bit_operations_comparison.append(total_BOPs)
        
# baseline model
total_BOPs = 0
# total_BOPs += bop_calculator(41, 41, 32, 32, 0)
total_BOPs += bop_calculator(41*140, 140, 32, 32, 0)
total_BOPs += bop_calculator(140*100, 100, 32, 32, 0)
total_BOPs += bop_calculator(100*40, 40, 32, 32, 0)
print(total_BOPs)
for i in range(4):
    bit_operations_comparison.append(total_BOPs)

In [ ]:
# Plotting BOPs
plt.figure()
q4_bitop = bit_operations_comparison[0:4]
q8_bitop = bit_operations_comparison[4:8]
q12_bitop = bit_operations_comparison[8:12]
q16_bitop = bit_operations_comparison[12:16]
baseline_model_BOPs = bit_operations_comparison[16:20]
plt.plot(sparsity_list, q4_bitop, 'x-', label='Quantized bits width = 4', color='blue')
plt.plot(sparsity_list, q8_bitop, 'x-', label='Quantized bits width = 8', color='green')
plt.plot(sparsity_list, q12_bitop, 'x-', label='Quantized bits width = 12', color='red')
plt.plot(sparsity_list, q16_bitop, 'x-', label='Quantized bits width = 16', color='black')
plt.plot(sparsity_list, baseline_model_BOPs, '-.', label='Baseline Model', color='orange')
plt.ylabel('BOPs [10e9]')
plt.xlabel('Sparsity')
plt.legend()
plt.savefig(f'{data_path}/Figures/PQ/BOPs.svg', bbox_inches='tight', format='svg', dpi=1200)
plt.show()

In [ ]:
plt.figure()
plt.scatter(q4_bitop, q4, color='blue', label='Quantized bits width = 4')
plt.scatter(q8_bitop, q8, color='green', label='Quantized bits width = 8')
plt.scatter(q12_bitop, q12, color='red', label='Quantized bits width = 12')
plt.scatter(q16_bitop, q16, color='black', label='Quantized bits width = 16')
plt.ylabel('MSE')
plt.xlabel('BOPs [10e8]')
plt.legend()
plt.savefig(f'{data_path}/Figures/PQ/BOPs_vs_Performance.svg', bbox_inches='tight', format='svg', dpi=1200)
plt.show()

In [ ]:
eval_pruned_model, size, qmodel_stripped = prune_and_quantize(0.6, 8)

In [ ]:
print(eval_pruned_model)
print(size)
qmodel_stripped.get_weights()

In [ ]:
# for key, values in holder.items():
#     print(key)
#     print(values['type'][2])
#     print(values['number_of_operations'])

In [ ]:
# print_qstats(qmodel)
# signed multiplication

# CDF plot

In [ ]:
df = pd.read_csv(f'{data_path}/Data/10000_initial_data_testing.csv')
# dataset
dataset = df.values
# input power and channels statuses
X = dataset[:,0:N_channels+1]
Y = dataset[:,N_channels+1:2*N_channels+1]

xScaler = preprocessing.MinMaxScaler()
yScaler = preprocessing.MinMaxScaler()

xScaler.fit(X)
yScaler.fit(Y)

xTrainNorm = xScaler.transform(X)
yTrainNorm = yScaler.transform(Y)

start_time = time.time()
y_predicted = qmodel_stripped.predict(xTrainNorm)
y_predicted_inverted = yScaler.inverse_transform(y_predicted)
prediction_time = time.time() - start_time
print(prediction_time)

In [ ]:
def CDF_plot(x, y, e_type):
    plt.figure()
    mse_ecdf = plt.plot(x,y, marker='.', linestyle='none', color='blue')
    plt.xlabel(f'{e_type} [dB]')
    plt.ylabel('Probability')
    plt.savefig(f'{data_path}/Figures/PQ/CDF_plot_QP_{e_type}.svg', bbox_inches='tight', format='svg', dpi=1200)
    plt.show()
    
mean_squared_error_store = []
mean_absolute_error_store = []
for i in range(10000):
    mean_squared_error_store.append(mean_squared_error(y_predicted_inverted[i], Y[i]))
    mean_absolute_error_store.append(mean_absolute_error(y_predicted_inverted[i], Y[i]))
x_mse = np.sort(mean_squared_error_store)
x_mae = np.sort(mean_absolute_error_store)
y = np.arange(1, len(x_mse)+1)/len(x_mse)
print('90th percentile mse = ', x_mse[9000],'dB')
print('90th percentile mae = ', x_mae[9000],'dB')
CDF_plot(x_mse, y, 'MSE')
CDF_plot(x_mae, y, 'MAE')

# Bar plot

In [ ]:
def barplot(index):
    y_actual = Y[index] * X[index][1:]
    y_predicted = y_predicted_inverted[index] * X[index][1:]
    
    labels = [i for i in range(1,41)]

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(18,5))
    # plt.figure(figsize=(25,5))
    rects1 = ax.bar(x - width/2, y_actual, width, label='Actual')
    rects2 = ax.bar(x + width/2, y_predicted, width, label='Predicted')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('GSNR [dB]', fontsize=15)
    ax.set_xlabel('Channel Number', fontsize=15)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    # ax.set_yticklabels(labels, fontsize=12)
    ax.legend(fontsize=15, loc='upper right')

    fig.tight_layout()

    plt.savefig(f'{data_path}/Figures/PQ/pruned_quantized_model_bar_chart_{index}.svg', bbox_inches='tight', format='svg', dpi=1200)
    plt.show()

barplot(2500)
barplot(5100)
barplot(7500)